# KaKao API를 이용하여 좌표->행정동 추출
* https://wooiljeong.github.io/python/kakao_local_api/
* https://github.com/henewsuh/KAKAO-map-API

In [11]:
import pandas as pd
import requests 
import sys
import json
import datetime 

In [12]:
def json_request(url='', encoding='utf-8', success=None, error=lambda e: print('%s : %s' % (e, datetime.now()), file=sys.stderr)):
    headers = {'Authorization': 'KakaoAK {}'.format(APP_KEY)}
    resp = requests.get(url, headers=headers)
    # print('%s : success for request [%s]' % (datetime.now(), url))
    return resp.text


def reverse_geocode(longitude, latitude):
    # 파라미터 최적화하여 url 생성
    url = '%s?x=%s&y=%s' %(URL, longitude, latitude)
    # json request
    try:
        # print('try')
        json_req = json_request(url=url)
        json_data = json.loads(json_req)
        json_doc = json_data.get('documents')[0]
        json_name = json_doc.get('address_name')
        json_code = json_doc.get('code')
    except:
        # print('nan')
        json_name = 'NaN'
        json_code = 'NaN'
    return json_name, json_code

 
def get_address(x,y):
    address = []        
    json_name, json_code = reverse_geocode(x, y) 
    address.append(json_name)
    return address # 전처리 함수에서 주소 리스트 받아서 데이터프레임에 추가


def get_code(x,y):
    code = []        
    json_name, json_code = reverse_geocode(x, y) 
    code.append(json_code)
    return code # 전처리 함수에서 행정구역코드 리스트 받아서 데이터프레임에 추가

In [18]:
APP_KEY = 'dc725f4d966ac26498c7388d7746eaaa' 
URL = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'

df_all = pd.read_csv('입지선정후보군데이터.csv', encoding='cp949') 
df = df_all[['분류', '시설명', 'x좌표', 'y좌표']] 

for i in range(len(df)): 
    x_crd = float(df.loc[i, ['x좌표']])
    y_crd = float(df.loc[i, ['y좌표']])
    address = get_address(x_crd, y_crd)
    code = get_code(x_crd, y_crd)
    df.loc[i, 'ADDRESS'] = address
    df.loc[i, 'CODE'] = code
    
    if i%100 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0 번째 경위도 좌표계에 대한 정보 불러오는 중...
100 번째 경위도 좌표계에 대한 정보 불러오는 중...
200 번째 경위도 좌표계에 대한 정보 불러오는 중...
300 번째 경위도 좌표계에 대한 정보 불러오는 중...
400 번째 경위도 좌표계에 대한 정보 불러오는 중...
500 번째 경위도 좌표계에 대한 정보 불러오는 중...
600 번째 경위도 좌표계에 대한 정보 불러오는 중...
700 번째 경위도 좌표계에 대한 정보 불러오는 중...
800 번째 경위도 좌표계에 대한 정보 불러오는 중...
900 번째 경위도 좌표계에 대한 정보 불러오는 중...
1000 번째 경위도 좌표계에 대한 정보 불러오는 중...
1100 번째 경위도 좌표계에 대한 정보 불러오는 중...
1200 번째 경위도 좌표계에 대한 정보 불러오는 중...
1300 번째 경위도 좌표계에 대한 정보 불러오는 중...
1400 번째 경위도 좌표계에 대한 정보 불러오는 중...
1500 번째 경위도 좌표계에 대한 정보 불러오는 중...
1600 번째 경위도 좌표계에 대한 정보 불러오는 중...
1700 번째 경위도 좌표계에 대한 정보 불러오는 중...
1800 번째 경위도 좌표계에 대한 정보 불러오는 중...
1900 번째 경위도 좌표계에 대한 정보 불러오는 중...
2000 번째 경위도 좌표계에 대한 정보 불러오는 중...
2100 번째 경위도 좌표계에 대한 정보 불러오는 중...
2200 번째 경위도 좌표계에 대한 정보 불러오는 중...
2300 번째 경위도 좌표계에 대한 정보 불러오는 중...
2400 번째 경위도 좌표계에 대한 정보 불러오는 중...
2500 번째 경위도 좌표계에 대한 정보 불러오는 중...
2600 번째 경위도 좌표계에 대한 정보 불러오는 중...
2700 번째 경위도 좌표계에 대한 정보 불러오는 중...


In [24]:
df = df[['분류', '시설명', 'x좌표', 'y좌표', 'ADDRESS']]
df = df.rename(columns={'ADDRESS': '행정동'})

In [32]:
def hangjungdong(x):
    return x.split()[-1]

df['행정동'] = df['행정동'].apply(hangjungdong)

In [31]:
print(df.행정동.nunique())
print(df.행정동.unique())

287
['석관동' '장위동' '하월곡동' '삼선동1가' '종암동' '상월곡동' '정릉동' '아현동' '서교동' '쌍문동' '창동'
 '신원동' '명륜3가' '천연동' '홍제동' '신당동' '신림동' '와룡동' '보문동7가' '보문동1가' '반포동' '암사동'
 '안암동2가' '돈암동' '길음동' '남가좌동' '영등포동' '방이동' '사직동' '시흥동' '신수동' '월계동' '대흥동'
 '홍은동' '북가좌동' '광장동' '구로동' '도봉동' '상계동' '창전동' '공덕동' '도화동' '방학동' '연남동' '개봉동'
 '성산동' '신도림동' '고척동' '북아현동' '영천동' '방배동' '대치동' '오류동' '양재동' '공항동' '서초동'
 '명륜4가' '내곡동' '가양동' '개포동' '일원동' '신사동' '사당동' '상도동' '흑석동' '대방동' '봉천동' '본동'
 '동작동' '도곡동' '우면동' '잠원동' '신월동' '방화동' '역삼동' '염곡동' '염창동' '내발산동' '화곡동' '등촌동'
 '명륜2가' '면목동' '중화동' '연건동' '논현동' '상수동' '상암동' '합정동' '청담동' '독산동' '장충동2가'
 '성수동1가' '목동' '중계동' '고덕동' '수서동' '자곡동' '용강동' '잠실동' '옥인동' '현석동' '효창동' '후암동'
 '망원동' '자양동' '진관동' '상일동' '마장동' '신공덕동' '삼성동' '신정동' '응암동' '불광동' '사근동' '공릉동'
 '송파동' '압구정동' '오금동' '세곡동' '한남동' '응봉동' '하계동' '염리동' '번동' '안암동5가' '구의동' '갈현동'
 '창신동' '녹번동' '당인동' '대조동' '동교동' '답십리동' '마곡동' '마포동' '망우동' '가산동' '어진동' '중동'
 '역촌동' '필운동' '풍납동' '항동' '구산동' '문배동' '이촌동' '이문동' '중곡동' '증산동' 'NaN' '장안동'
 '신대방동' '성북동' '부암동' '갈월동' '궁동' '수유동' '도림동' '만리동1가' '신영동

In [38]:
#df.to_csv('입지선정후보군데이터.csv', encoding='cp949')

### 그 외 추가 작업은 엑셀

In [43]:
a = pd.read_csv("pos00006m.csv", encoding='utf-8')
a.to_csv("encoding.csv", encoding='cp949')